In [1]:
### LIBRARIES

# OPENBIS
from pybis import Openbis
o = Openbis()
# Use this code to reconnect in case your openBIS session expires and you an error on the previous step.
sessionToken = "pierrecu-230119132124122x14B5340A4EC9CA1F57666CFAC85F229A"
o.set_token(token=sessionToken)

Session is no longer valid. Please log in again.


In [2]:
# BUILT-IN
import os
import random
import shutil

# THIRD PART
from zipfile import ZipFile

# CUSTOM

In [3]:
# MAIN PARAMS

project = o.get_project('/MATERIALS/NRMD')
raw_modes = ['BLED',
             'SDCD',
             'LSLD',
             'STIM'
            ]
population = o.get_objects(type = 'PARTICIPANT')

parent_proj = o.get_object('/METHODS/NRMD/PROJ_1') #related project and funding information in Methods
parent_sensor = o.get_object('/METHODS/NRMD/SENS_1') #related sensors information in Methods
parent_stim = o.get_object('/METHODS/NRMD/STIM_3') #related stimulation information in Methods

get_samples posting request
get_samples got response. Delay: 0.0838479995727539
get_samples got JSON. Delay: 0.007565975189208984
get_samples after result mapping. Delay: 0.0011181831359863281
_sample_list_for_response before parsing JSON
_sample_list_for_response got response. Delay: 0.0009090900421142578
_sample_list_for_response computing result.
_sample_list_for_response computed result. Delay: 0.0006902217864990234
get_samples computed final result. Delay: 0.004529237747192383


In [4]:
for mode in raw_modes:
    ''' 
    # LOOP THROUGH EACH RAW DATA MODE
    try:
    ''' 
    print(f"looking for raw data mode: {mode}")
    uploads = o.get_sample(f'/MATERIALS/NRMD/NRMD_UPLD_RAWD_{mode}')
    uploads_dsets = uploads.get_datasets()

    # LOOP THROUGH EACH UPLOADED DATASET
    for _dset in uploads_dsets:

        # LOCALLY DOWNLOAD THE ZIP FILES
        _files = _dset.file_list
        _dset.download(files=_files, destination = f'./{mode}/', wait_until_finished=True)

        # LIST FILES
        download_path = os.path.join('.',f'{mode}',_dset.code,'original','OPENBIS_UPLOADS')
        downloaded_files = os.listdir(download_path)
        print(f"    temporary download of {len(downloaded_files)} zipped datasets:\n\n    {downloaded_files}\n")

        # UNZIP DATASETS
        for _f in downloaded_files:
            if '.zip' in _f:
                # get metadata
                the_file = _f.split("/")[-1]
                the_name = the_file.split("_")
                the_id = the_name[0]
                the_phase = the_name[1]
                the_session = the_name[2]
                the_type = _dset.type
                the_typecode = the_type.code
                the_download_path = os.path.join(download_path,the_file)
                the_extract_path = os.path.join(download_path,f"{the_id}_{the_session}")
                for pop in population:
                    if pop.props('participant.id') == str(int(the_id)):
                        the_parent = pop
                        break
                    else:
                        the_parent = None
                print(f"file:{_f}\n   - the_id:{the_id}\n   - the_phase:{the_phase}\n   - the_session:{the_session}\n   - the_type:{the_type}")

                # unzip datasets
                with ZipFile(the_download_path, 'r') as zipObj:
                    zipObj.extractall(the_extract_path)
                print(f"   - extracted to: {the_extract_path}")

                os.remove(the_download_path)
                print(f"   - removed from: {the_download_path}")

                the_collection = o.get_collection(f'/MATERIALS/NRMD/NRMD_{the_type}')
                print(f"   - getting data collection: {the_collection.code}")

                # get or create a data recording mode sample for the id_session
                try:
                    the_sample = o.get_sample(f'/MATERIALS/NRMD/NRMD_{the_type}_{the_id}_{the_phase}_{the_session}')
                    print(f"   - getting existing data recording mode: {the_sample.code}")
                except Exception as e:
                    
                    
                    
                    
                    the_sample = o.new_sample(type = 'DATA_RECORDING_MODE',
                                              code = f'NRMD_{the_type}_{the_id}_{the_phase}_{the_session}',
                                              project = project.identifier,
                                              space = 'MATERIALS',
                                              collection = f'/MATERIALS/NRMD/{the_collection.code}',
                                              props = {'data_recording_mode.data_recording_mode':the_typecode.split('_')[1],
                                                       'data_recording_mode.session': the_session,
                                                       'data_recording_mode.phase': the_phase,
                                                       'data_recording_mode.participant': the_parent.identifier,
                                                      },
                                            )
                    the_sample.save()
                    the_sample.parents = [the_parent,parent_proj,parent_sensor,parent_stim]
                    the_sample.save()
                    print(f"   - getting newly created data recording mode: {the_sample.code}")

                # create a new dataset
                the_dset_name = the_file.split('.')[0]
                the_subpath = os.listdir(the_extract_path)[0]
                
                if os.path.isdir(the_subpath):
                    the_subfiles = [os.path.join(the_extract_path,the_subpath,_z) for _z in os.listdir(os.path.join(the_extract_path,the_subpath))]
                else:
                    the_subfiles = [os.path.join(the_extract_path,_z) for _z in os.listdir(the_extract_path)]

                the_dset = o.new_dataset(type = the_typecode, 
                                         sample = f'/MATERIALS/NRMD/{the_sample.code}',
                                         files = the_subfiles,
                                         props = {f'{the_typecode.lower()}.name':the_dset_name,
                                                  f'{the_typecode.lower()}.session': the_session,
                                                  f'{the_typecode.lower()}.phase': the_phase,
                                                  f'{the_typecode.lower()}.participant': the_parent.identifier,
                                                 },
                                        )
                the_dset.save()
                #the_dset.parents = [the_parent,]
                #the_dset.save()
                print(f"   - dataset created: {the_dset.code}")

                shutil.rmtree(the_extract_path)
                print(f"   - removed from: {the_extract_path}")


        # check the cleaned download and extraction repo
        print(f"\ncleaned repo:\n      {os.listdir(download_path)}\n")
        # delete the uploaded datasets
        _dset.delete(reason = 'raw dataset creation routine completed.')
'''           
    except Exception as e:
        print(f"    {e}")
''' 

looking for raw data mode: BLED
Files downloaded to: ./BLED/20230119163925430-1815
    temporary download of 36 zipped datasets:

    ['3807_P001_S003_BBT-E32-AAB052-2022-11-01_12-16-17.zip', '4466_P001_S007_BBT-E32-AAB052-2022-12-15_16-34-22.zip', '4466_P001_S000_BBT-E32-AAB052-2022-10-17_08-15-28.zip', '3807_P001_S009_BBT-E32-AAB052-2022-11-16_14-09-06.zip', '3807_P001_S019_BBT-E32-AAB052-2022-12-09_08-53-11.zip', '3807_P001_S005_BBT-E32-AAB052-2022-11-02_10-41-23.zip', '4466_P001_S006_BBT-E32-AAB052-2022-12-15_15-54-05.zip', '3807_P001_S023_BBT-E32-AAB052-2022-12-14_16-47-23.zip', '3807_P001_S001_BBT-E32-AAB052-2022-10-26_16-33-57.zip', '4466_P001_S003_BBT-E32-AAB052-2022-12-07_07-53-45.zip', '3807_P001_S015_BBT-E32-AAB052-2022-11-29_10-55-02.zip', '3807_P001_S014_BBT-E32-AAB052-2022-11-29_08-46-29.zip', '4466_P001_S002_BBT-E32-AAB052-2022-11-17_07-56-52.zip', '3807_P001_S012_BBT-E32-AAB052-2022-11-18_13-33-04.zip', '3807_P001_S006_BBT-E32-AAB052-2022-11-02_14-04-08.zip', '3807_P001

'           \n    except Exception as e:\n        print(f"    {e}")\n'

In [5]:
# quickly update existing samples with new parents
#the_collection = o.get_collection('/MATERIALS/NRMD/NRMD_RAWD_STIM')
#for _s in the_collection.get_samples():
#    _s.parents += [parent_proj, parent_sensor, parent_stim]
#    _s.save()